## Sample generation
#### load trained model -> generate samples

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
IGNORE_NOTE_VELOCITY = True
TARGET_IS_SEQUENCE = True
MAX_WINDOW_SIZE = 100
PLOT_SEEDS = True

import numpy as np
from tensorflow import keras

INPUT_SIZE = 128 if IGNORE_NOTE_VELOCITY else 256
# [batch_size, sequence_len, input_dim]

model_name = 'm1560126804'

input_path = '/content/drive/My Drive/datasets/'
input_file_name = f'heck_512_stacked_all/{model_name}.h5'

model = keras.models.load_model(input_path + input_file_name)

### Generate seed

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt


# helpers
scales = {
    "natural_minor": [0, 2, 3, 5, 7, 8, 10, 12],
    "harmonic_minor": [0, 2, 3, 5, 7, 8, 11, 12],
    "melodic_minor": [0, 2, 3, 5, 7, 9, 11, 12],
    "major": [0, 2, 4, 5, 7, 9, 11, 12],
    "dorian": [0, 2, 4, 5, 7, 9, 10, 12],
    "mixolydian": [0, 2, 4, 5, 7, 9, 10, 12],
    "pentatonic": [0, 3, 5, 7, 10, 12],
}

def get_random_scale():
    return random.choice(list(scales.values()))

def get_random_notes_from_random_scale(base_note, n_notes=1):
    scale = get_random_scale()
    two_octave_scale = scale + [-x for x in scale]
    return [base_note + random.choice(two_octave_scale) for _ in range(n_notes)]

def create_noise_adder(noise_gen, noise_scale=0.5, **nkwargs):
    def inner(seed_gen, **kwargs):
        return lambda length, batch_size: np.clip((
            noise_scale * noise_gen(length, batch_size, **nkwargs)
            + seed_gen(length, batch_size, **kwargs)), 0, 1
        )
    return inner

# na = create_noise_adder(band_noise_seed)
# x = na(multi_note_simult_harmonic_seed)(length=128, batch_size=16)
def plot_seq(x):
    if len(x.shape) == 3:
        nrows = x.shape[0] // 4
        fig, axs = plt.subplots(nrows=nrows, ncols=4, figsize=(12, nrows * 3),
                                subplot_kw={'xticks': [], 'yticks': []})
        for ax, x_ in zip(axs.flat, x):
            ax.imshow(x_.T[::-1, :])
    else:
        plt.imshow(x[0].T[::-1, :])
    plt.tight_layout()
    res = plt.gcf()
    plt.show()
    return res    

In [ ]:
def random_noise_seed(length=MAX_WINDOW_SIZE, batch_size=16):
    return np.random.random((batch_size, length, INPUT_SIZE)) * 0.5

In [ ]:
def zero_seed(length=MAX_WINDOW_SIZE, batch_size=16):
    return np.zeros((batch_size, length, INPUT_SIZE))

In [ ]:
# notes in middle range have highier values
# taken from normal distribution for each "hand"
# mu1 = 76, mu2 = 48, std = 12
def band_noise_seed(length=MAX_WINDOW_SIZE, batch_size=16):
    mu1 = 76
    mu2 = 48
    std = 12
    hand1 = np.exp(-(np.arange(128) - mu1) ** 2 / (2 * std ** 2))
    hand2 = np.exp(-(np.arange(128) - mu2) ** 2 / (2 * std ** 2))
    hands = hand1 + hand2
    randomness = np.random.random(((batch_size, length, INPUT_SIZE))) * 0.5
    return randomness * hands

In [ ]:
def single_note_seed(length=MAX_WINDOW_SIZE, batch_size=16):
    res = np.zeros((batch_size, length, INPUT_SIZE))
    for i in range(batch_size):
        note = int(np.random.normal(72, 16))
        note = np.clip(note, 0, 127)
        res[i, :, note] = 1
    return res

In [ ]:
def single_note_short_seed(length=MAX_WINDOW_SIZE, batch_size=16, max_note_length=25):
    res = np.zeros((batch_size, length, INPUT_SIZE))
    for i in range(batch_size):
        note = int(np.random.normal(72, 16))
        note = np.clip(note, 0, 127)
        note_on = np.random.randint(length)
        note_off = np.random.randint(note_on, min(note_on + max_note_length, length))
        res[i, note_on:note_off, note] = 1
    return res

In [ ]:
def multi_note_seed(length=MAX_WINDOW_SIZE, batch_size=16, num_notes=5):
    res = np.zeros((batch_size, length, INPUT_SIZE))
    for j in range(batch_size):
        notes = np.random.normal(72, 16, size=num_notes).round().astype(int)
        notes = np.clip(notes, 0, 127)
        note_change_times = np.sort(np.random.choice(np.arange(length), size=num_notes + 1))
        for i, note in enumerate(notes):
            res[j, note_change_times[i]:note_change_times[i+1], note] = 1
    return res

def multi_note_harmonic_seed(length=MAX_WINDOW_SIZE, batch_size=16, num_notes=5):
    res = np.zeros((batch_size, length, INPUT_SIZE))
    base_note = 72
    for j in range(batch_size):
        notes = get_random_notes_from_random_scale(base_note, num_notes)
        note_change_times = np.sort(np.random.choice(np.arange(length), size=num_notes + 1))
        for i, note in enumerate(notes):
            res[j, note_change_times[i]:note_change_times[i+1], note] = 1
    return res

In [ ]:
def multi_note_simult_seed(length=MAX_WINDOW_SIZE, batch_size=16, num_notes=15, max_note_length=25):
    res = np.zeros((batch_size, length, INPUT_SIZE))
    for j in range(batch_size):
        notes = np.random.normal(72, 16, size=num_notes).round().astype(int)
        notes = np.clip(notes, 0, 127)
        for i, note in enumerate(notes):
            note_on = np.random.randint(length)
            note_off = np.random.randint(note_on, min(note_on + max_note_length, length))
            res[j, note_on:note_off, note] = 1
    return res

def multi_note_simult_harmonic_seed(length=MAX_WINDOW_SIZE, batch_size=16, num_notes=15, max_note_length=25):
    res = np.zeros((batch_size, length, INPUT_SIZE))
    base_note = 72
    for j in range(batch_size):
        notes = get_random_notes_from_random_scale(base_note, num_notes)
        for i, note in enumerate(notes):
            note_on = np.random.randint(length)
            note_off = np.random.randint(note_on, min(note_on + max_note_length, length))
            res[j, note_on:note_off, note] = 1
    return res

### All seeds

In [ ]:
bna = create_noise_adder(band_noise_seed)
rna = create_noise_adder(random_noise_seed)

seed_generators = {
    "random_noise_seed": random_noise_seed,
    "zero_seed": zero_seed,
    "band_noise_seed": band_noise_seed,
    "single_note_seed": single_note_seed,
    "single_note_short_seed": single_note_short_seed,
    "multi_note_seed": multi_note_seed,
    "multi_note_harmonic_seed": multi_note_harmonic_seed,
    "multi_note_simult_seed": multi_note_simult_seed,
    "multi_note_simult_harmonic_seed": multi_note_simult_harmonic_seed,
    "single_note_seed_noise": rna(single_note_seed),
    "single_note_seed_band noise": bna(single_note_seed),
    "single_note_short_seed_noise": rna(single_note_short_seed),
    "single_note_short_seed_band noise": bna(single_note_short_seed),
    "multi_note_harmonic_seed_noise": rna(multi_note_harmonic_seed),
    "multi_note_harmonic_seed_band noise": bna(multi_note_harmonic_seed),
    "multi_note_simult_harmonic_seed_noise": rna(multi_note_simult_harmonic_seed),
    "multi_note_simult_harmonic_seed_band noise": bna(multi_note_simult_harmonic_seed),
}

In [ ]:
%matplotlib notebook
fig, axs = plt.subplots(nrows=len(seed_generators.items()), ncols=4, figsize=(12, 51),
                                subplot_kw={'xticks': [], 'yticks': []})
for i, (name, gen) in enumerate(seed_generators.items()):
    x = gen(length=128, batch_size=4)
    for j, x_ in enumerate(x):
        if j ==0:
            axs.flat[4 * i + j].set_title(name, size=10)
        axs.flat[4 * i + j].imshow(x_.T[::-1, :])

# plt.tight_layout()
plt.show()

### Generate sequence

In [ ]:
# at default 1 frame = 0.025s
SEQUENCE_LENGTH = 600
SEED_LENGTH = 599

def generate(seed, seq_length, window_len=SEED_LENGTH):
    # sequence shape is [batch_size, sequence_length, input_size]
    x = seed
    accum = [seed]
    for i in range(seq_length - window_len):
        print(i, end=',')
        res = model.predict(x).round()
        if TARGET_IS_SEQUENCE:
            # next input consists input
            # and last slice of result
            # (then limited to window size)
            x = np.concatenate([x, res[:, -1:, :]], axis=1)[:, -MAX_WINDOW_SIZE:, :]
            accum.append(res[:, -1:, :])
        else:
            # next input consists of previous input
            # with result frame attatched to end
            # (then limited to window size)
            x = np.concatenate([x, res[:, np.newaxis, :]], axis=1)[:, -MAX_WINDOW_SIZE:, :]
            accum.append(res[:, np.newaxis, :])
    return np.concatenate(accum, axis=1).round()

In [ ]:
%matplotlib notebook

name, seed_gen = list(seed_generators.items())[0]

samples = [generate(seed_gen(length=SEED_LENGTH, batch_size=1), SEQUENCE_LENGTH) for _ in range(4)]
samples = np.concatenate(samples, axis=0)
print(samples.shape)
plot_seq(samples).show()

### Save to numpy format

In [ ]:
from datetime import datetime

def save_numpy_midi(output_path, res):
    import numpy as np
    from scipy import sparse
    if '.csv' in output_path:
        np.savetxt(output_path, res, delimiter=",")
        print(f'Saved to: {output_path}')
    elif '.npy' in output_path:
        np.save(output_path, res)
        print(f'Saved to: {output_path}')
    elif '.npz' in output_path:
        res_sparse = sparse.coo_matrix(res)
        sparsity_factor = 1 - (res_sparse.getnnz() / res.size)
        sparse.save_npz(output_path, res_sparse)
        print(
            f'Saved to: {output_path}, {int(100 * sparsity_factor)}% sparsity')

tim = str(datetime.now()).replace(' ', '_').replace(':', '_')
output_path = '/content/drive/My Drive/datasets/heck_512_stacked_all/samples/'
for i, sample in enumerate(samples):    
    output_file_name = f'{model_name}_{name}_{tim}_{i}.npz'
    save_numpy_midi(output_path + output_file_name, sample)